In [1]:
import pandas as pd 
import numpy as np

## 端午节的淘宝粽子交易
### 问题

(1) 请删除最后一列为缺失值的行，并求所有在杭州发货的商品单价均值。

(2) 商品标题带有“嘉兴”但发货地却不在嘉兴的商品有多少条记录?

(3) 请按照分位数将价格分为“高、较高、中、较低、低”5 个类别，再将 类别结果插入到标题一列之后，最后对类别列进行降序排序。

(4) 付款人数一栏有缺失值吗?若有则请利用上一问的分类结果对这些缺失 值进行合理估计并填充。

(5) 请将数据后四列合并为如下格式的 Series:商品发货地为 ××，店铺为 ××，共计 ×× 人付款，单价为 ××。

(6) 请将上一问中的结果恢复成原来的四列。

In [179]:
df_1 = pd.read_csv('./Testdata/端午粽子数据.csv')
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4403 entries, 0 to 4402
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   标题      4403 non-null   object
 1    价格     4403 non-null   object
 2   付款人数    4332 non-null   object
 3   店铺      4403 non-null   object
 4   发货地址    4400 non-null   object
dtypes: object(5)
memory usage: 172.1+ KB


In [180]:
#(1) 请删除最后一列为缺失值的行，并求所有在杭州发货的商品单价均值。
df_zz = df_1.copy().dropna(axis=0,subset=['发货地址 '])#4400条，以下问题均以此表为基础

pd.to_numeric(df_zz[df_zz['发货地址 '].str.contains('杭州')][' 价格'],errors='coerce').mean().round(decimals=2)

81.1

In [181]:
#(2) 商品标题带有“嘉兴”但发货地却不在嘉兴的商品有多少条记录?
df_zz[(df_zz['标题'].str.contains('嘉兴')) &(~df_zz['发货地址 '].str.contains('嘉兴'))].shape[0]

1032

In [182]:
#(3) 请按照分位数将价格分为“高、较高、中、较低、低”5 个类别，再将 类别结果插入到标题一列之后，最后对类别列进行降序排序。

df_zz.loc[:,' 价格'] = pd.to_numeric(df_zz.loc[:,' 价格'],errors='coerce')

q = [df_zz.loc[:,' 价格'].quantile(i) for i in [0,.2,.4,.6,.8,1]];q[-1] += 1
df_zz.loc[:,'价格类别'] = pd.cut(df_zz[' 价格'],q,right=False,labels=['低','较低','中','较高','高'])


In [183]:
df_zz.loc[:,['标题','价格类别', ' 价格', '付款人数', '店铺', '发货地址 ']].sort_values(by='价格类别')


,标题,价格类别,价格,付款人数,店铺,发货地址
3666,嘉兴五芳斋粽子紫米栗蓉粽板栗栗子甜粽子五谷素粽真空袋装美味,低,11.0,162人付款,三珍斋食品专营店,江苏 南京
1623,白米粽子新鲜湖南甜粽原味小粽子农家手工散装绿豆红枣碱水清水粽,低,20.9,181人付款,乐家_stone,湖南 常德
1053,【9只特价】新鲜蛋黄肉粽 嘉兴粽子特产端午节批发团购早餐粽手工,低,20.8,1358人付款,怡品三清旗舰店,浙江 杭州
3743,三珍斋嘉兴粽子大肉粽子蛋黄肉粽豆沙蜜枣端午节散装批发鲜肉棕子,低,19.9,35人付款,三珍斋乌镇专卖,浙江 嘉兴
1311,现做新鲜粽子咸蛋黄肉粽甜粽端午节散装嘉兴蜜枣豆沙大批發礼盒装,低,9.9,5000+人付款,徽老九美食,安徽 亳州
...,...,...,...,...,...,...
3548,锦华粽子瑞阳祥礼端午节礼盒新鲜嘉兴肉素六粒粽蛋卷组合送人礼品,高,110.6,36人付款,北京咏逢礼信商贸有限公司,北京
3523,印象昆韵新品昆曲文化创意端午节胚芽糙米手工肉粽子端阳礼品礼盒,高,156.0,21人付款,印象昆韵,江苏 苏州
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,高,129.0,6人付款,五芳斋官方旗舰店,浙江 嘉兴
538,五芳斋粽子蛋黄鲜肉粽豆沙粽栗子肉棕子新鲜散装批发团购嘉兴肉粽,NaN,NaN,1.0万+人付款,五芳斋官方旗舰店,浙江 嘉兴


In [184]:
#(4) 付款人数一栏有缺失值吗?若有则请利用上一问的分类结果对这些缺失 值进行合理估计并填充。

#非缺失值有 4329/4400
df_z4 = df_zz.copy()
df_z4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4400 entries, 0 to 4402
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   标题      4400 non-null   object  
 1    价格     4398 non-null   float64 
 2   付款人数    4329 non-null   object  
 3   店铺      4400 non-null   object  
 4   发货地址    4400 non-null   object  
 5   价格类别    4398 non-null   category
dtypes: category(1), float64(1), object(4)
memory usage: 210.7+ KB


In [185]:
#用同价格类别的付款人数均值（不考虑100余条类似1.0万+、5000+等较大数值。）
grouped = df_z4.groupby('价格类别')

for name,group in grouped:
    df_z4.loc[group.index,'付款人数'] = str(int(pd.to_numeric(group['付款人数'].astype(str).str[:-3],errors='coerce').mean()))+'人付款'

df_z4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4400 entries, 0 to 4402
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   标题      4400 non-null   object  
 1    价格     4398 non-null   float64 
 2   付款人数    4400 non-null   object  
 3   店铺      4400 non-null   object  
 4   发货地址    4400 non-null   object  
 5   价格类别    4398 non-null   category
dtypes: category(1), float64(1), object(4)
memory usage: 370.7+ KB


In [186]:
#(5) 请将数据后四列合并为如下格式的 Series:商品发货地为 ××，店铺为 ××，共计 ×× 人付款，单价为 ××。
df_z5 = df_z4.astype(str)
df_str1 = ('商品发货地为'+df_z5['发货地址 ']+'，店铺为'+df_z5['店铺']+'，共计'+df_z5['付款人数']+'，单价为'+df_z5[' 价格']+'。')
df_str1.head()

0    商品发货地为浙江 嘉兴，店铺为五芳斋官方旗舰店，共计139人付款，单价为129.0。
1            商品发货地为上海，店铺为天猫超市，共计357人付款，单价为44.0。
2     商品发货地为浙江 嘉兴，店铺为五芳斋官方旗舰店，共计291人付款，单价为89.9。
3       商品发货地为北京，店铺为稻香村食品旗舰店，共计139人付款，单价为138.0。
4         商品发货地为浙江 嘉兴，店铺为城城喂食猫，共计540人付款，单价为3.8。
dtype: object

In [187]:
#(6) 请将上一问中的结果恢复成原来的四列。

df_res = df_str1.str.extract(r'商品发货地为(?P<发货地址>[\w]+\s*[\w]+)，店铺为(?P<店铺>[\w]+)，共计(?P<付款人数>[\w]+\+*[\w]+)，单价为(?P<价格>[\d]+\.*[\d]+)。')
df_res.head()


,发货地址,店铺,付款人数,价格
0,浙江 嘉兴,五芳斋官方旗舰店,139人付款,129.0
1,上海,天猫超市,357人付款,44.0
2,浙江 嘉兴,五芳斋官方旗舰店,291人付款,89.9
3,北京,稻香村食品旗舰店,139人付款,138.0
4,浙江 嘉兴,城城喂食猫,540人付款,3.8


## 墨尔本每日最低温度 
### 问题 

(1) 剔除国庆节、五一劳动节和每月第一个周一，求每月的平均最低气温。 

(2) 季节指数是一种对于周期性变化序列的特征刻画。记数据集中第 k 年 平均最低气温为 TYk(k = 1, ..., 10)，第 k 年第 j 个月平均最低气温为 。请按照如上定义，计算 12 个 ∑ TMkj TM (j = 1, ..., 12)，定义 S = k kj j 月的季节指数 Sj。

(3) 移动平均法是一种时间序列的常见平滑方式，可分为 k 期移动平均和 k 期中心移动平均，都使用了某一时刻及其周围的数据对该时刻的数据进行平滑修正。

In [188]:
df_2 = pd.read_csv('./Testdata/墨尔本温度数据.csv')#墨尔本1981-1990共十年最低气温数据
df_2['Date'] = pd.to_datetime(df_2['Date'])
df_2 = df_2.set_index('Date',drop=True)
df_2.index

DatetimeIndex(['1981-01-01', '1981-01-02', '1981-01-03', '1981-01-04',
               '1981-01-05', '1981-01-06', '1981-01-07', '1981-01-08',
               '1981-01-09', '1981-01-10',
               ...
               '1990-12-22', '1990-12-23', '1990-12-24', '1990-12-25',
               '1990-12-26', '1990-12-27', '1990-12-28', '1990-12-29',
               '1990-12-30', '1990-12-31'],
              dtype='datetime64[ns]', name='Date', length=3650, freq=None)

In [189]:
# (1) 剔除国庆节、五一劳动节和每月第一个周一，求每月的平均最低气温。
# 1999年9月,国务院改革出台新的法定休假制度,每年国庆节、春节和“五一”法定节日加上倒休,全国放假7天。
# 假设本题中国庆节、五一劳动节特指10.1、5.1两天
d = pd.date_range(start='1981-1-1',end='1990-12-31')
mask1 = (d.weekday == 0) & (0< d.day) & (d.day < 8)
mask10 = (d.month==10)&(d.day==1) 
mask5 = (d.month==5)&(d.day==1) 
holidays = d[mask1].append(d[mask5]).append(d[mask10]).sort_values()
weekmask = 'Sun Mon Tue Wed Thu Fri Sat'

df_2_new = df_2.copy().reindex(pd.bdate_range(start='1981-1-1',end='1990-12-31',freq='C',weekmask = weekmask,holidays=holidays))
#求每月的平均最低气温
df_2_new.resample('M').mean().head()

,Temp
1981-01-31,17.776667
1981-02-28,17.637037
1981-03-31,13.330000
1981-04-30,12.210345
1981-05-31,9.551724


In [191]:
#(2) 季节指数,月均值/年均值
TY   = df_2.resample('Y').mean().sum()
TMkj = df_2.resample('M').mean()

for i in range(12):
    j = i+1
    print('S{}='.format(j),(TMkj[TMkj.index.month==j].sum()/TY).values[0].round(decimals=4))

S1= 1.3447
S2= 1.3758
S3= 1.3031
S4= 1.0815
S5= 0.8827
S6= 0.6511
S7= 0.5987
S8= 0.706
S9= 0.8031
S10= 0.9223
S11= 1.1165
S12= 1.2394


In [192]:
#(3) 移动平均
#(a) 求原序列的 5 期移动平均序列(自身与前四个的均值)
df_2.rolling(window=5,min_periods=-5).sum()/5

,Temp
Date,
1981-01-01,4.14
1981-01-02,7.72
1981-01-03,11.48
1981-01-04,14.40
1981-01-05,17.56
...,...
1990-12-27,13.08
1990-12-28,13.02
1990-12-29,13.72


In [193]:
#(b) 原序列的 5 期中心移动平均序列。
df_2.rolling(window=5,min_periods=-5).sum().shift(-2)/5
#原序列的 6 期中心移动平均序列。
(df_2.shift(-3)/2+df_2.rolling(window=5,min_periods=-5).sum().shift(-2)+df_2.shift(3)/2)/6

,Temp
Date,
1981-01-01,NaN
1981-01-02,NaN
1981-01-03,NaN
1981-01-04,16.858333
1981-01-05,16.408333
...,...
1990-12-27,13.575000
1990-12-28,14.058333
1990-12-29,NaN



## 2016 年 8 月上海市摩拜单车骑行记录
### 问题

(1) 平均而言，周末单天用车量比工作日单天用车量更大吗?

(2) 工作日每天的高峰时间段大致为上午 7:30 至 9:30、下午 17:00 至 19:00， 请问 8 月里早高峰骑行记录量(以 start_time 为准)高于晚高峰的有 几天?

(3) 请给出在所有周五中(以 start_time 为准)，记录条数最多的那个周五 所在的日期，并在该天内分别按 30 分钟、2 小时、6 小时统计摩拜单车 使用时间的均值。

(4) 请自行搜索相关代码或调用库，计算每条记录起点到终点的球面距离。

(5) 摩拜单车的骑行结束时间是以电子锁关闭的记录时间为准，但有时候用 户会忘记关锁，导致骑行时间出现异常。同时，正常人的骑行速度往往 大致落在一个合理的区间，请结合上一问中的球面距离和骑行起始、结 束时间，找出潜在的异常骑行记录。

(6) 由于路线的曲折性，起点到终点的球面距离往往不能充分反应行程长 度，请利用 track 列的路线坐标数据，计算估计实际骑行距离，并重新 仿照上一问的方法找出可能的异常记录。


In [317]:
df_3 = pd.read_csv('Testdata/摩拜单车数据.csv')
df_3.start_time = pd.to_datetime(df_3.start_time)
df_3.end_time = pd.to_datetime(df_3.end_time)
df_3 = df_3.set_index('start_time',drop=False)

In [318]:
#（1) 平均而言，周末单天用车量比工作日单天用车量更大吗?
print('周末单天用车量比工作日单天用车量更大吗?')
print(df_3.resample('D').count().mean()[0]>df_3.resample('B').count().mean()[0])

#只需判断是否有：【工作日+周末】日均用车>【工作日】日均用车

周末单天用车量比工作日单天用车量更大吗?
False


In [319]:
#(2) 工作日每天的高峰时间段大致为上午 7:30 至 9:30、下午 17:00 至 19:00， 请问 8 月里早高峰骑行记录量(以 start_time 为准)高于晚高峰的有 几天?
highday=[]
for i in range(31):
    day = '2016-8-{}'.format(i+1)
    if(pd.to_datetime(day).dayofweek>4):continue#跳过周末
    morning   = df_3[(df_3.index>='{} 07:30'.format(day))&( df_3.index<='{} 09:30'.format(day))].count()[0]
    afternoon = df_3[(df_3.index>='{} 17:00'.format(day))&( df_3.index<='{} 19:00'.format(day))].count()[0]
    if(morning>afternoon):
        highday.append(day)
print(len(highday),highday)


3 ['2016-8-4', '2016-8-29', '2016-8-30']


In [353]:
#(3) 请给出在所有周五中(以 start_time 为准)，记录条数最多的那个周五 所在的日期，
Fri = df_3[df_3.index.dayofweek==4]['orderid'].resample('D').count().idxmax()
print('条数最多的那个周五是',Fri.date())

df_3_Fri = df_3[str(Fri.date())].copy()
df_3_Fri['usetime'] = (df_3_Fri.end_time-df_3_Fri.start_time).dt.total_seconds()#转为绝对秒数求均值
#并在该天内分别按 30 分钟、2 小时、6 小时统计摩拜单车 使用时间的均值。
display(
    df_3_Fri['usetime'].resample('30min').mean().apply(lambda x:pd.Timedelta(seconds = int(x))).head(),
    df_3_Fri['usetime'].resample('2h'   ).mean().apply(lambda x:pd.Timedelta(seconds = int(x))).head(),
    df_3_Fri['usetime'].resample('6h'   ).mean().apply(lambda x:pd.Timedelta(seconds = int(x))).head(),
)


条数最多的那个周五是 2016-08-26


start_time
2016-08-26 00:00:00   00:18:56
2016-08-26 00:30:00   00:26:42
2016-08-26 01:00:00   00:13:27
2016-08-26 01:30:00   00:15:30
2016-08-26 02:00:00   00:17:17
Freq: 30T, Name: usetime, dtype: timedelta64[ns]

start_time
2016-08-26 00:00:00   00:19:57
2016-08-26 02:00:00   00:19:18
2016-08-26 04:00:00   00:13:33
2016-08-26 06:00:00   00:11:31
2016-08-26 08:00:00   00:11:35
Freq: 2H, Name: usetime, dtype: timedelta64[ns]

start_time
2016-08-26 00:00:00   00:17:47
2016-08-26 06:00:00   00:11:39
2016-08-26 12:00:00   00:16:00
2016-08-26 18:00:00   00:21:39
Freq: 6H, Name: usetime, dtype: timedelta64[ns]

In [398]:
#（4）起点终点球面距离
from geopy.distance import geodesic
df_3_dist = df_3.copy()
#df_3_dist.head()

In [399]:
#计算起始点距离，单位m
df_3_dist['se_dist'] = df_3_dist.apply(lambda x: geodesic((x.start_location_y,x.start_location_x),(x.end_location_y,x.end_location_x)).m,axis=1)



In [400]:
#(5)请结合上一问中的球面距离和骑行起始、结 束时间，找出潜在的异常骑行记录。
df_3_dist['usetime'] = (df_3_dist.end_time-df_3_dist.start_time).dt.total_seconds()#转为绝对秒数求均值
df_3_dist['se_speed'] = df_3_dist.apply(lambda x:x.se_dist/x.usetime,axis=1)

In [401]:
print('mean',df_3_dist.se_speed.mean())
for q in [0.95,0.75,0.5,0.25,0.1,0.05,0.04,0.03,0.02,0.01,0.001]:
    print(q*100,'分位数',df_3_dist.se_speed.quantile(q))

#可能忘记锁车的：
df_3_dist[df_3_dist.se_speed<df_3_dist.se_speed.quantile(0.05)].head()

mean 1.8288589220755427
95.0 分位数 2.9496406260729646
75.0 分位数 2.297319181282499
50.0 分位数 1.874990122656719
25.0 分位数 1.3956780310929044
10.0 分位数 0.8143215529850231
5.0 分位数 0.4438671243387195
4.0 分位数 0.34814530845338243
3.0 分位数 0.2563034915875663
2.0 分位数 0.1741136367495188
1.0 分位数 0.09947094328371195
0.1 分位数 0.031518593078144516


,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,se_dist,usetime,se_speed
start_time,,,,,,,,,,,,,
2016-08-29 09:21:00,270836,63136,11361,2016-08-29 09:21:00,121.369,31.252,2016-08-29 13:43:00,121.376,31.233,"121.367,31.238#121.367,31.239#121.368,31.240#1...",2209.621048,15720.0,0.140561
2016-08-26 22:05:00,1802934,320611,1211,2016-08-26 22:05:00,121.504,31.305,2016-08-26 22:27:00,121.500,31.308,"121.499,31.308#121.500,31.308#121.501,31.308#1...",505.608255,1320.0,0.383037
2016-08-24 13:32:00,1630164,143066,11182,2016-08-24 13:32:00,121.436,31.328,2016-08-24 13:44:00,121.439,31.327,"121.436,31.328#121.437,31.328#121.437,31.329#1...",306.300607,720.0,0.425418
2016-08-27 21:52:00,1623489,151285,17526,2016-08-27 21:52:00,121.478,31.289,2016-08-27 22:14:00,121.476,31.292,"121.476,31.292#121.477,31.289#121.477,31.290#1...",383.276389,1320.0,0.290361
2016-08-03 22:08:00,748550,136503,7423,2016-08-03 22:08:00,121.483,31.237,2016-08-03 22:26:00,121.484,31.234,"121.482,31.235#121.483,31.234#121.483,31.235#1...",345.995048,1080.0,0.320366


In [424]:
#(6) 利用 track 列的路线坐标数据，计算估计实际骑行距离，并重新仿照上一问的方法找出可能的异常记录。
track = '121.367,31.238#121.367,31.239#121.368,31.240#121.368,31.241#121.368,31.242#121.368,31.252#121.369,31.242#121.369,31.243#121.369,31.244#121.369,31.245#121.369,31.252#121.370,31.245#121.370,31.246#121.370,31.247#121.370,31.251#121.370,31.252#121.371,31.248#121.371,31.249#121.371,31.250#121.371,31.251'
def getdist(track):
    tlist=[]
    dist = 0
    for t in track.split('#'):
        for tt in t.split(','):
            (tlist.append(tt))
    for i in range(int(len(tlist)/2)-1):
        try:
            dist+=geodesic((tlist[i*2+1],tlist[i*2]),(tlist[i*2+3],tlist[i*2+2])).m
        except ValueError:
            dist+=0
    return dist
getdist(track)

6153.382508885842

In [425]:
df_3_dist['track_dist']=df_3_dist.track.apply(lambda x:getdist(x))

In [426]:
df_3_dist['track_speed'] = df_3_dist.apply(lambda x:x.track_dist/x.usetime,axis=1)

In [427]:
print('mean',df_3_dist.track_speed.mean())
for q in [0.95,0.75,0.5,0.25,0.1,0.05,0.04,0.03,0.02,0.01,0.001]:
    print(q*100,'分位数',df_3_dist.track_speed.quantile(q))

mean 4.717642405397278
95.0 分位数 11.119561310481727
75.0 分位数 5.6486331534791026
50.0 分位数 3.869183948181684
25.0 分位数 2.6515407909180317
10.0 分位数 1.377678636142472
5.0 分位数 0.6461082250002582
4.0 分位数 0.502248793681997
3.0 分位数 0.36700311152670406
2.0 分位数 0.2489091789769442
1.0 分位数 0.14214735328501782
0.1 分位数 0.03553589878562731


In [429]:
#可能忘记锁车的：
df_3_dist[df_3_dist.track_speed<df_3_dist.track_speed.quantile(0.05)]

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,se_dist,usetime,se_speed,track_dist,track_speed
start_time,,,,,,,,,,,,,,,
2016-08-29 09:21:00,270836,63136,11361,2016-08-29 09:21:00,121.369,31.252,2016-08-29 13:43:00,121.376,31.233,"121.367,31.238#121.367,31.239#121.368,31.240#1...",2209.621048,15720.0,0.140561,6153.382509,0.391437
2016-08-31 18:15:00,1563430,384828,945,2016-08-31 18:15:00,121.606,31.110,2016-08-31 18:58:00,121.570,31.115,"121.606,31.111#121.607,31.110#121.609,31.110#1...",3478.566745,2580.0,1.348282,432.445014,0.167614
2016-08-29 07:45:00,80889,335198,12380,2016-08-29 07:45:00,121.354,31.226,2016-08-29 08:28:00,121.356,31.225,"121.351,31.223#121.353,31.226#121.354,31.223#1...",220.465615,2580.0,0.085452,1205.743298,0.467342
2016-08-16 19:56:00,1504315,78807,185,2016-08-16 19:56:00,121.480,31.311,2016-08-16 20:04:00,121.484,31.305,"121.480,31.311#121.482,31.311",766.519391,480.0,1.596915,190.385816,0.396637
2016-08-12 22:02:00,1419609,106895,11697,2016-08-12 22:02:00,121.515,31.291,2016-08-12 22:17:00,121.537,31.301,"121.515,31.290#121.516,31.290",2369.929234,900.0,2.633255,95.214021,0.105793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-08-03 16:39:00,373681,85225,7484,2016-08-03 16:39:00,121.449,31.269,2016-08-03 16:57:00,121.432,31.277,"121.448,31.270#121.449,31.269",1845.992264,1080.0,1.709252,146.159845,0.135333
2016-08-24 20:25:00,1777023,313903,13647,2016-08-24 20:25:00,121.363,31.254,2016-08-24 20:41:00,121.364,31.255,"121.363,31.254#121.363,31.255#121.364,31.255#1...",146.169464,960.0,0.152260,463.165717,0.482464
2016-08-24 20:13:00,1774184,103791,15364,2016-08-24 20:13:00,121.530,31.324,2016-08-24 20:20:00,121.524,31.328,"121.528,31.324#121.529,31.324#121.530,31.324",723.055998,420.0,1.721562,190.359665,0.453237
